In [ ]:
import glob
import json

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from tqdm import tqdm

import sys
sys.path.insert(0, "../src")

from utils.utils import read_json

# Set variables

In [ ]:
INP_DIR = "../../datasets/caesar_pointclouds"
OUT_DIR = "../output/dataset_20230309-103305/"

NUM_POINTS = 20000

# Read input

In [ ]:
in_file_list = glob.glob(INP_DIR + "/**/*.ply", recursive=True)
len(in_file_list)

In [ ]:
avatar = in_file_list[0]
avatar

In [ ]:
mesh = o3d.io.read_triangle_mesh(avatar)
mesh.compute_vertex_normals()

avatar_pcd = mesh.sample_points_poisson_disk(number_of_points=NUM_POINTS)

# Sanity check

In [ ]:
out_file_list = glob.glob(OUT_DIR + "/**/*.png", recursive=True)

In [ ]:
im_path = out_file_list[12]
im_path

In [ ]:
img = o3d.io.read_image(im_path)

print(np.asarray(img).min())
print(np.asarray(img).max())
print(np.asarray(img).dtype)

In [ ]:
fig, axi = plt.subplots(1, 1, figsize=(15, 5))
cb_ = axi.imshow(img, cmap="gray")
fig.colorbar(cb_, ax=axi)

In [ ]:
json_files = glob.glob(OUT_DIR + "/**/*.json", recursive=True)

intrinsic = o3d.camera.PinholeCameraIntrinsic(
    **read_json(list(filter(lambda x: "intrinsic" in x, json_files))[0])
)
extrinsic = read_json(list(filter(lambda x: "extrinsic" in x, json_files))[0])["extrinsics"]
scale = read_json(list(filter(lambda x: "scale" in x, json_files))[0])["depth_scale"]

pcd = o3d.geometry.PointCloud.create_from_depth_image(
    img,
    #o3d.geometry.Image(a),
    intrinsic,
    extrinsic,
    depth_scale=scale,
    depth_trunc=4000.0
)

_, ind = pcd.remove_statistical_outlier(nb_neighbors=5, std_ratio=1.0)
#_, ind = pcd.remove_radius_outlier(nb_points=35, radius=20.0)
inlier_cloud = pcd.select_by_index(ind)

o3d.visualization.draw_geometries([
    pcd,
    #inlier_cloud
])